In [161]:
import pandas as pd
import math
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, accuracy_score

def input_filters(input_data_set, test_dataset, label_name, among_which_labels_list, based_list, k):
    input_data_1=pd.read_csv(input_data_set)
    label=label_name
    based=based_list
    input_data = input_data_1[based]
    input_data = input_data.reindex(columns=test_dataset.columns)
    input_data[label]=input_data_1[label]
    adf= input_data[input_data[label].isin(among_which_labels_list)]

    return adf, test_dataset


In [162]:
def convert_dataset_dict(input_data, test_data, based):
    k = 5
    count_based=input_data.shape[1]
    input_based_dict={}
    test_dict={}
    input_based_dict = input_data.to_dict(orient='list')
    test_dict = test_data.to_dict(orient='list')

    return input_based_dict, test_dict


In [163]:
def dist_formula1(input_based_dict, test_list_row, label):
    
    all_dist=[]
    length=[]
    for key,value in input_based_dict.items():
        l=input_based_dict[key]
        length.append(len(l))
    max_length=max(length)
    
    for j in range(max_length):
        dist=[]
        total=0
        i=0
        for key1,values in input_based_dict.items():
            if (i<=len(test_list_row) and key1!=label):
                value_in=values[j]
                value_test=test_list_row[i]
                d=((value_test-value_in)**2)
                dist.append(d)
                i=i+1
            else:
                break
                
        for num in dist:
            total += num
        all_dist.append(np.sqrt(total))
    input_based_dict.update({"Dist":all_dist})
    ls=pd.DataFrame.from_dict(input_based_dict, orient='columns')
    sorted_input = ls.sort_values(by='Dist')
    
    return sorted_input


In [164]:
def predict(k, data, label):
    top_val = data.iloc[:k,:][label].value_counts().to_dict()
    max_value = max(top_val, key=top_val.get)

    return max_value


In [165]:
def iter(input_based_dict, test_dict, label,k):
    length=[]
    for key1,value in test_dict.items():
        l=test_dict[key1]
        length.append(len(l))
    max_length=max(length)
    
    predicted=[]
    for i in range(max_length):
        list1=[]
        for key,value in test_dict.items():
            list1.append(value[i])
        pred=predict(k,dist_formula1(input_based_dict, list1, label), label)
        predicted.append(pred)
    test_dict["Predicted_value"]=predicted

    return predicted
    

In [166]:
def myKNN_classifier(input_data_set, test_dataset, label_name, among_which_labels_list, based_list, k):
    test=pd.read_csv(test_dataset)
    out1=input_filters(input_data_set, test, label_name, among_which_labels_list, based_list, k)
    out2=convert_dataset_dict(out1[0], out1[1], based_list)
    out3=iter(out2[0],out2[1], label_name,k)
    test["Predicted"]=out3
    test["Input"]=out1[0]["De"]
    
    return test


In [176]:
knn=myKNN_classifier("./admission-1.csv", "./admission-1 test.csv", 'De', ['admit', 'notadmit'], ["GPA","GMAT"], 7)
knn.head(84)

,GPA,GMAT,Predicted,Input
0,2.96,596,admit,admit
1,3.14,473,notadmit,admit
2,3.22,482,notadmit,admit
3,3.29,527,admit,admit
4,3.69,505,notadmit,admit
5,3.46,693,admit,admit
6,3.03,626,admit,admit
7,3.19,663,admit,admit
8,3.63,447,notadmit,admit
9,3.59,588,admit,admit


In [177]:
def confusion_matrix_and_accuracy(true_labels, predicted_labels):
    confusion = confusion_matrix(true_labels, predicted_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    return confusion, accuracy

confusion, accuracy = confusion_matrix_and_accuracy(knn['Input'], knn['Predicted'])
print("Confusion:\n", confusion)
print("\nAccuracy:", accuracy)


Confusion:
 [[24  7]
 [ 2 26]]

Accuracy: 0.847457627118644
